In [1]:
from __future__ import division
import numpy as np
from scipy import stats

In [4]:
from functools import partial

In [1]:
def online_changepoint_detection(data, hazard_func, observation_likelihood):
    maxes = np.zeros(len(data) + 1)
    
    R = np.zeros((len(data) + 1, len(data) + 1))
    R[0, 0] = 1
    
    for t, x in enumerate(data):
        # Evaluate the predictive distribution for the new datum under each of
        # the parameters.  This is the standard thing from Bayesian inference.
        predprobs = observation_likelihood.pdf(x)
        
        # Evaluate the hazard function for this interval
        H = hazard_func(np.array(range(t+1)))
       
        # Evaluate the growth probabilities - shift the probabilities down and to
        # the right, scaled by the hazard function and the predictive
        # probabilities.
        R[1:t+2, t+1] = R[0:t+1, t] * predprobs * (1-H)
        
        # Evaluate the probability that there *was* a changepoint and we're
        # accumulating the mass back down at r = 0.
        R[0, t+1] = np.sum( R[0:t+1, t] * predprobs * H)
        
        # Renormalize the run length probabilities for improved numerical
        # stability.
        R[:, t+1] = R[:, t+1] / np.sum(R[:, t+1])
        
        # Update the parameter sets for each possible run length.
        observation_likelihood.update_theta(x)
    
        maxes[t] = R[:, t].argmax()
    return R, maxes

In [1]:
def constant_hazard(lam, r):
    return 1/lam * np.ones(r.shape)

In [1]:
class StudentT:
    def __init__(self, alpha, beta, kappa, mu):
        self.alpha0 = self.alpha = np.array([alpha])
        self.beta0 = self.beta = np.array([beta])
        self.kappa0 = self.kappa = np.array([kappa])
        self.mu0 = self.mu = np.array([mu])

    def pdf(self, data):
        return stats.t.pdf(x=data, 
                           df=2*self.alpha,
                           loc=self.mu,
                           scale=np.sqrt(self.beta * (self.kappa+1) / (self.alpha *
                               self.kappa)))

    def update_theta(self, data):
        muT0 = np.concatenate((self.mu0, (self.kappa * self.mu + data) / (self.kappa + 1)))
        kappaT0 = np.concatenate((self.kappa0, self.kappa + 1.))
        alphaT0 = np.concatenate((self.alpha0, self.alpha + 0.5))
        betaT0 = np.concatenate((self.beta0, self.beta + (self.kappa * (data -
            self.mu)**2) / (2. * (self.kappa + 1.))))
            
        self.mu = muT0
        self.kappa = kappaT0
        self.alpha = alphaT0
        self.beta = betaT0

In [13]:
data = [1,3,0,0]

In [14]:
a,b = online_changepoint_detection(data, partial(constant_hazard, 250), StudentT(10, .03, 1, 0))

In [15]:
print(a)

[[1.00000000e+00 4.00000000e-03 4.00000000e-03 4.00000000e-03
  4.00000000e-03]
 [0.00000000e+00 9.96000000e-01 2.85265914e-12 3.24324646e-01
  9.75320585e-03]
 [0.00000000e+00 0.00000000e+00 9.96000000e-01 1.37562376e-12
  9.36245130e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 6.71675354e-01
  1.12177227e-13]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  5.00016638e-02]]


In [21]:
StudentT(10, .03, 1, 0).pdf(data[1])*(249/250)

array([9.03908841e-20])

In [22]:
StudentT(10, .03, 1, 0).pdf(data[1])*(1/250)

array([3.63015599e-22])

In [24]:
3.63015599e-22/(9.03908841e-20+3.63015599e-22)

0.004000000002247837

# Blog 

#### Pre-requisites

Marginal Probability $P(x) = \sum_{y}P(x,y)$<br>

Conditional Probabilty $P(x | y) = \frac{P(x,y)}{P(y)}$ 

Bayes Theorem
>$P(\theta | D ) \propto P(D | \theta) * P(\theta)$ \
$posterior = likelihood * prior$\
<br>
Lets try to define some distributions,\
$D \propto pdf(\theta)$\
$\theta \propto pdf(\gamma)$\
Now, $\gamma$ is a hyperprior, which can either by a r.v or can be constant\
So when we recieve data, we update our beliefs i.e hyperprior.\
<br> 
Now when $\gamma$ is not a r.v (i.e it does not follow any distribution), then:<br>
$P(\theta | D,\gamma ) \propto P(D | \theta, \gamma) * P(\theta | \gamma)$\
$P(\theta | D,\gamma ) \propto P(D | \theta) * P(\theta)$ , assuming D is independent of every thing when conditioned on $\theta$,  this can go wrong is the data points are not iid, for example time series data with significant auto-correlation.\
<br>
When $\gamma$ also follows a distribution, then:<br> 
$P(\theta | D,\gamma ) \propto P(D | \theta, \gamma)*P(\theta | \gamma)$\
$P(\theta | D,\gamma )*P(\gamma) \propto P(D | \theta, \gamma)*P(\theta | \gamma) * P(\gamma)$\
$P(\theta,\gamma | D ) \propto P(D | \theta, \gamma)*P(\theta | \gamma) * P(\gamma)$ , conditional probabilty\
$P(\theta,\gamma | D ) \propto P(D | \theta)*P(\theta | \gamma) * P(\gamma)$ , assuming D is independent of every thing when conditioned on $\theta$\

#### Whats the use?<br>
Economics, genetics, anomoly detection, stock market

#### Defn 1 :
t : some time \
$x_t$ : observed data at time t<br>
$x_{1:t}$ : $x_1,x_2,....x_{t-1}, x_t$<br>
$x_t^r$ : data points in current run\
$r_t$ : run length at time t ( number of time points from last change point)<br>
Change point is when $r_t = 0$

#### What we want to achieve?
We want to figure out if the recieved data point is a change point, i.e $P(r_t | x_{1:t})$\
We want to be able to predict the next data point, i.e $P(x_{t+1} | x_{1:t})$

#### Assumption 1 :<br>
Data can be divided into several partitions, data points in each partition are iid's, i.e $x_t^r \propto pdf(\eta)$\
$x_t^r$ is conditionally independent of everything given $\eta$\
$r_t$ is conditionally independent of everything given $r_{t-1}$ -

### Predictive Version

This is the predictive version<br>

$P(x_{t+1}|x_{1:t}) = \sum_{r_t}P(x_{t+1}|r_t,x_{1:t})*P(r_t | x_{1:t})$ , this comes from conditional and marginal probability<br>

Since $x_{t+1}$ only depend on $x_t^r$<br>
$P(x_{t+1}|x_{1:t}) = \sum_{r_t}P(x_{t+1}|r_t,x_t^r)*P(r_t | x_{1:t})$

$P(r_t | x_{1:t}) = \frac{P(r_t , x_{1:t})}{P(x_{1:t})}$, from bayes theorem

Hence to have predictive version in place we need 3 things, $P(r_t , x_{1:t}), P(x_{1:t}), P(x_{t+1}|r_t,x_t^r)$

### Changepoint Detection

$P(r_t,x_{1:t}) = \sum_{r_{t-1}}P(r_t,x_{1:t}, r_{t-1})$ *this again from marginal distribution, P(A) = $\sum_{\forall b}P(A,B)$*<br>
$P(r_t,x_{1:t}) = \sum_{r_{t-1}}P(r_t,x_{1:t-1}, r_{t-1},x_t)$ *just split $x_{1:t}$*<br>
$P(r_t,x_{1:t}) = \sum_{r_{t-1}}P(r_t,x_t | x_{1:t-1}, r_{t-1})*P(x_{1:t-1}, r_{t-1})$ *this is just P(A,B) = P(A|B)P(B)*<br>


$P(r_t,x_{1:t}) = \sum_{r_{t-1}}P(x_t | r_t, x_{1:t-1}, r_{t-1})*P(r_t|x_{1:t-1}, r_{t-1})*P(x_{1:t-1}, r_{t-1})$ , using chain rule<br>
$P(r_t,x_{1:t}) = \sum_{r_{t-1}}P(r_t|r_{t-1})*P(x_t | x_{1:t-1}, r_{t-1})*P(x_{1:t-1}, r_{t-1})$ , *$r_t$ only depends on $r_{t-1}$*<br>
$P(r_t,x_{1:t}) = \sum_{r_{t-1}}P(r_t|r_{t-1})*P(x_t | x_{t-1}^{r_{t-1}}, r_{t-1})*P(x_{1:t-1}, r_{t-1})$ ,

### References 

https://nbviewer.jupyter.org/github/hildensia/bayesian_changepoint_detection/blob/master/Example%20Code.ipynb
https://arxiv.org/pdf/0710.3742.pdf
http://gregorygundersen.com/blog/2019/08/13/bocd/#:~:text=Bayesian%20online%20changepoint%20detection%20is,modularity%20and%20efficient%20parameter%20estimation
https://github.com/hildensia/bayesian_changepoint_detection/blob/master/bayesian_changepoint_detection/
https://en.wikipedia.org/wiki/Forward_algorithm#Algorithm